# 1) Wetterdaten API call Geosphere + MongoDB
FÜr unser Projekt haben wir uns für einerseits die Wetterdaten entschieden und mussten somit einen API call machen. Wir haben uns für die Station Hohe Warte, 1190 Wien mit der ID 11035 entschieden über den Zeitraum November 2025 - Dezember 2025.

Diese Station liegt relativ zentral und kann somit über grob über Wiens Wetterverhältnisse aussagen.
Folgende Parameter schauen wir uns näher an:
RR - Niederschlag
TL - Lufttemperatur
P - Luftdruck
FF - Windgeschwindigkeit
SO - Sonnenscheindauer
RF - Relative Feuchte

Danach haben wir diese Daten in der MongoDB gespeichert

In [1]:
!pip install requests pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 159.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 181.5 kB/s eta 0:00:00a 0:00:01


In [3]:
import requests
import pandas as pd
from pymongo import MongoClient, errors
import io

# --- KONFIGURATION ---
DB_NAME = "big_data_austria"        #Datenbank
COLLECTION_NAME = "Station_HW_25" #Tabelle (Collection)

# API Setup
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/tawes-v1-10min"
params = {
    "parameters": ["RR", "TL", "P", "FF", "SO", "RF"],
    "start": "2025-11-01T00:00",
    "end": "2025-12-31T00:00",
    "station_ids": "11035",
    "output_format": "geojson"
}

try:
    # 1. Daten von GeoSphere laden
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # DataFrame erstellen
    timestamps = data.get("timestamps", [])
    props = data["features"][0]["properties"]["parameters"]
    storage_dict = {"timestamp": timestamps}
    for p in params["parameters"]:
        if p in props:
            storage_dict[p] = props[p]["data"]
    
    df = pd.DataFrame(storage_dict)
    df['station_id'] = "HW_25"
    mongo_docs = df.to_dict(orient="records")

    # --- 2. MONGODB VERBINDUNG & INDEX ---

    client = MongoClient('mongodb://mongodb:27017')
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # UNIQUE INDEX ERSTELLEN, damit timestamp passt. 1 bedeutet aufsteigend sortiert.
    print("Erstelle Unique Index für 'timestamp'...")
    collection.create_index([("timestamp", 1)], unique=True)

    # --- 3. DATEN SPEICHERN MIT DUPLIKAT-CHECK ---
    if mongo_docs:
        success_count = 0
        duplicate_count = 0
        
        for doc in mongo_docs:
            try:
                collection.insert_one(doc)
                success_count += 1
            except errors.DuplicateKeyError:
                duplicate_count += 1
        
        print(f"--- ABSCHLUSSBERICHT ---")
        print(f"Datenbank: {DB_NAME} | Collection: {COLLECTION_NAME}")
        print(f"Neu gespeichert: {success_count}")
        print(f"Übersprungene Duplikate: {duplicate_count}")
    
except Exception as e:
    print(f"Fehler: {e}")

Erstelle Unique Index für 'timestamp'...
--- ABSCHLUSSBERICHT ---
Datenbank: big_data_austria | Collection: Station_HW_25
Neu gespeichert: 8641
Übersprungene Duplikate: 0
